<a href="https://colab.research.google.com/github/AngelJmzIsd/Iris/blob/main/Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  _____      _       _____           _                    
# |_   _|    (_)     |  __ \         | |                   
#   | |  _ __ _ ___  | |__) |_ _  ___| | ____ _  __ _  ___ 
#   | | | '__| / __| |  ___/ _` |/ __| |/ / _` |/ _` |/ _ \
#  _| |_| |  | \__ \ | |  | (_| | (__|   < (_| | (_| |  __/
# |_____|_|  |_|___/ |_|   \__,_|\___|_|\_\__,_|\__, |\___|
#                                                __/ |     
#                                               |___/      

Este modulo tiene el objetivo de hacer más fácil el proceso de limpieza, seleccion de features y evaluacion de modelos de ML para el reto de Ternium.

---
Fue creado en base a un interés personal por acelerar y agilizar los procesos de limpieza, entrenamiento y evaluación de modelos, así como de poder escribir un codigo mucho más comprensible y conciso, al mismo tiempo que preciso.


---
EXPLICACION DE LOS NOMBRES: 

El primer arbol de decisiones que realice fue analizando el dataset de las flores Iris, un conjunto de datos utilizado ampliamente para aprender ciencia de datos, modelos de clasificacion, y basicamente, introducirse en el mundo de la Inteligencia Artificial.

---

Por el momento se contemplan dos clases:

 - **Versicolor:** creada en base a un dataframe de pandas, las funciones miembre tienen el objetivo de transformar, manipular y limpiar la base de datos, más no de entrenar o evaluar los modelo.


 - **Setosa**: esta tiene el objetivo de 

NOTA: El 15 de junio de 2022 a las 00:14 se cambio el nombre de "Versicolor" a

"Iris", los nombres de las clases pasan a ser "Versicolor" (antes "Dataset") y 

"Setosa" (antes "Evaluador"). 

In [ ]:
#Importar librerias principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#Seleccion de features
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


#Transformacion de datos
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


#Modelos de clasificacion
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.svm import SVC
from sklearn import svm
from sklearn.pipeline import make_pipeline



#EVALUACION DE MODELOS
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import plot_roc_curve


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc


In [ ]:

#   _____
#  / ____| |               \ \    / /          (_)         | |           
# | |    | | __ _ ___ ___   \ \  / /__ _ __ ___ _  ___ ___ | | ___  _ __ 
# | |    | |/ _` / __/ __|   \ \/ / _ \ '__/ __| |/ __/ _ \| |/ _ \| '__|
# | |____| | (_| \__ \__ \    \  /  __/ |  \__ \ | (_| (_) | | (_) | |   
#  \_____|_|\__,_|___/___/     \/ \___|_|  |___/_|\___\___/|_|\___/|_|   
                                                                      

'''
################################################################################
################################################################################
################################################################################
################################################################################

Es la clase Versicolor que en realidad es lo mismo que un dataframe, solo que
le puse funciones para hacer la limpieza y la descripcion de forma mas rapida
#y el codigo más entendible

################################################################################
################################################################################
################################################################################
################################################################################
'''

class Versicolor:

  #Dataframe que guarda el estado original del objeto, aunque se contempla
  #eliminarlo pronto, por el uso de la memoria

  dfRespaldo=pd.DataFrame()

  nombreTarget=[]



  #Df = objeto dataframe de pandas
  #Nombre = etiqueta o alias tipo string para asignarle una descripcion
  #Por ejemplo alias="datos crudos, sin limpiar"

  def __init__(self,dataframe, nombre):
    self.dataframe=pd.DataFrame()
    self.dataframe=dataframe.copy()
    self.dfRespaldo=self.dataframe.copy()
    self.nombre=nombre


  #Funcion para obtener los nombres de las variables que son de tipo object y
  #que posiblemente son categoricas

  def setNombreTarget(self,nombreColumnaTarget):
    self.nombreTarget.append(nombreColumnaTarget)
    return None

  def getNombresNumericas(self):
    camposNumericos=[]
    campos=self.dataframe.columns
    n=len(campos)

    for i in range(0,n):
      nombreCampo=campos[i]
      tipoCampo=self.dataframe[nombreCampo].dtype.name
      if (tipoCampo == 'int64' or tipoCampo=='float64'):
        camposNumericos.append(nombreCampo)

    return camposNumericos



  def getNombresNoNumericas(self):
    camposNoNumericos=[]
    campos=self.dataframe.columns
    n=len(campos)

    for i in range(0,n):
      nombreCampo=campos[i]
      tipoCampo=self.dataframe[nombreCampo].dtype.name
      if (tipoCampo == 'object'):
        camposNoNumericos.append(nombreCampo)

    return camposNoNumericos


#Regresa el datagrame a su estado original, mediante l 
  def reset(self):
    self.dataframe=self.dfRespaldo.copy()



################################################################################
################################################################################
################################################################################
################################################################################
#
#                           LIMPIEZA DE DATOS
#
################################################################################
################################################################################
################################################################################
################################################################################


# Funcion para obtener nombres columnas redundantes con un solo 
#valor o todos null
  def getColumnasVacias(self):
    columnas=list(self.dataframe.columns)
    redundantes=[]
    n=len(columnas)

    for i in range (0,n):
      columna=str(columnas[i])
      valoresUnicos=len(pd.unique(self.dataframe[columna]))
      if (valoresUnicos <= 1): redundantes.append(columna)

    return redundantes


  #Funcion para quitar un conjunto de columnas del dataset
  def quitarColumnas(self,nombresColumnas):
    self.dataframe=self.dataframe.drop(columns=nombresColumnas)
    return 0



  # Primero calcula los limites superiores e inferiores de cada columna, luego
  # quita la
  def contarAtipicosPorColumnas(self,nombresColumnas):
    #Crear un nuevo dataframe solo con las columnas deseadas
    df=pd.DataFrame()
    for col in nombresColumnas:
      df[col]=self.dataframe[col].copy()

    #Calcular las metricas para cada uno de las cosas esas

    q1=list(df.quantile(0.25))
    q3=list(df.quantile(0.75))
    q1=pd.Series(q1)
    q3=pd.Series(q3)

    rango=q3-q1
    limSup=q3+1.5*rango
    limInf=q1-1.5*rango

    rango=list(rango)
    limSup=list(limSup)
    limInf=list(limInf)

    cantidadAtipicos=[]
    n=len(nombresColumnas)

    for i in range (0,n):
      columna=nombresColumnas[i]

      dfAux=pd.DataFrame()
      dfAux= self.dataframe[      (self.dataframe[columna] < limInf[i]) or (self.dataframe[columna] > limSup[i])     ]

      cantidadAtipicos.append(dfAux.shape[0])

    return cantidadAtipicos
  #Recorre cada columna y quita la fila si encuentra un atipico
  def quitarAtipicosPorColumnas(self,nombresColumnas):
    return 0

  

################################################################################
################################################################################
################################################################################
  #PRIORITARIOS
################################################################################
################################################################################
################################################################################



  def contarFaltantesColumnas(self,nombresColumnas,porcentaje):
    cantidadFaltantes=[]
    n=len(nombresColumnas)

    for i in range(0,n):
      columna=nombresColumnas[i]
      nulos=self.dataframe[columna].isna().sum()

      if(porcentaje==True): nulos = (100*nulos/self.dataframe.shape[0])
      cantidadFaltantes.append(nulos)
    
    return cantidadFaltantes

  def contarFaltantesColumnasCustom(self,nombresColumnas,porcentaje,customNa):
    cantidadFaltantes=[]
    n=len(nombresColumnas)

    for i in range(0,n):
      columna=nombresColumnas[i]
      nulos=self.dataframe[ self.dataframe[columna]==customNa]
      faltantes=nulos.sum()
      #nulos=self.dataframe[columna].isna().sum()

      if(porcentaje==True): faltantes = (100*faltantes/self.dataframe.shape[0])
      cantidadFaltantes.append(faltantes)
    
    return cantidadFaltantes

  def getColumnasVaciasPorcentajeMenorIgualA(self,porcentaje):
    columnas=list(self.dataframe.columns)
    porcentajeFaltantes=self.contarFaltantesColumnas(columnas,True)

    vaciasPorcentaje=[]
    n=len(columnas)

    for i in range(0,n):
      if (porcentajeFaltantes[i]<=porcentaje):
        vaciasPorcentaje.append(columnas[i])
    return vaciasPorcentaje

  def getColumnasVaciasPorcentajeMayorA(self,porcentaje):
    columnas=list(self.dataframe.columns)
    porcentajeFaltantes=self.contarFaltantesColumnas(columnas,True)

    vaciasPorcentaje=[]
    n=len(columnas)

    for i in range(0,n):
      if (porcentajeFaltantes[i]>porcentaje):
        vaciasPorcentaje.append(columnas[i])
    return vaciasPorcentaje

  #def porcentajeFaltantesColumnas(self,nombresColumnas):
    #return 0


  #medida = "moda","promedio","mediana"
  def reemplazarFaltantesColumnas(self,nombresColumnas,medida):
    n=len(nombresColumnas)

    if (medida=="promedio"):
      for i in range(0,n):
        columna=nombresColumnas[i]
        self.dataframe[columna]=self.dataframe[columna].fillna(self.dataframe[columna].mean())

    if (medida=="mediana"):
      for i in range(0,n):
        columna=nombresColumnas[i]
        self.dataframe[columna]=self.dataframe[columna].fillna(self.dataframe[columna].median())

    if (medida=="moda"):
      for i in range(0,n):
        columna=nombresColumnas[i]
        moda=list(self.dataframe[columna].mode().squeeze())[0]
        self.dataframe[columna]=self.dataframe[columna].fillna(moda)
    return 0


  def reemplazarFaltantesColumnasCustom(self,nombresColumnas,medida,customNa):

    n=len(nombresColumnas)

    if (medida=="promedio"):
      for i in range(0,n):
        columna=nombresColumnas[i]
        #self.dataframe[columna].fillna(self.dataframe[columna].mean(),inplace = True)

    if (medida=="mediana"):
      for i in range(0,n):
        columna=nombresColumnas[i]
        #self.dataframe[columna].fillna(self.dataframe[columna].median(),inplace = True)

    if (medida=="moda"):
      for i in range(0,n):
        columna=nombresColumnas[i]
        moda=list(self.dataframe[columna].mode().squeeze())[0]
        self.dataframe[columna] = np.where(self.dataframe[columna] == customNa, moda,self.dataframe[columna])
    return 0

  def exportarCsv(self,ruta,codificacion):
    if (codificacion==0):
      self.dataframe.to_csv(ruta,encoding='latin-1')
    else:
      self.dataframe.to_csv(ruta,encoding=codificacion)
    return None



################################################################################
################################################################################
################################################################################
#
#       FUNCIONES PARA SELECCION DE CARACTERISTICAS
#
################################################################################
################################################################################
################################################################################


  def setFeatures():
    return 0

  def setTarget():
    return 0

  def getFeatures():
    return 0

  def getTarget():
    return 0


  def getTargetBinarioDefecto(self,nombreColumnaTarget,nombreDefecto):
    columna=self.dataframe[nombreColumnaTarget].copy()
    df=pd.DataFrame()
    df[nombreDefecto]=np.where(self.dataframe[nombreColumnaTarget] == nombreDefecto, 1, 0)
    return df[nombreDefecto].copy()

  def getKbestChi2DefectoIndividual(self,nombreColumnaTarget,nombreDefecto,numeroFeatures):
    
    
    #features=self.dataframe.copy()
    target=self.getTargetBinarioDefecto(nombreColumnaTarget,nombreDefecto)
    nombresNumericas=self.getNombresNumericas()
    dfNumericas=pd.DataFrame()

    #Crear nuevo dataframe con puras columnas numericas y al final agregarle
    #la variable Target

    for nombre in nombresNumericas:
      dfNumericas[nombre]=self.dataframe[nombre].copy()
    dfNumericas['class']=target.copy()

    M=len(dfNumericas.columns)
    array = dfNumericas.values

    X = array[:,0:M-1]
    Y = array[:,M-1]

    test=SelectKBest(score_func=chi2, k=numeroFeatures)
    fit = test.fit(X, Y)
    puntajes=fit.scores_
    features = fit.transform(X)

    campos=list(dfNumericas.columns[0:-1])
    scores=list(puntajes)

    indices=[]

    for i in range(0,len(campos)+1):
      indices.append(i)

    nombreCamposSeleccionados=[]
    scoreSeleccionados=[]
    posicionCamposSeleccionados=[]
    rank=[]

    #AHORA VOY A HACER UN CICLO FOR QUE ENCUENTRE EL INDICE DEL MAXIMO VALOR EN LA LISTA DE SCORES Y LO VAYA GUARDANDO Y REPITA ESTO 1O VECES


    for i in range(0,numeroFeatures):
      scoreMaximo=0
      indiceMaximo=0
      nombreCampoMaximo=""

      scoreMaximo=max(scores)
      indiceMaximo=scores.index(scoreMaximo)

      posicionEnOriginal=indices[indiceMaximo]

      #Guardar los datos de los maximos

      scoreSeleccionados.append(scoreMaximo)
      posicionCamposSeleccionados.append(indices[indiceMaximo])
      nombreCamposSeleccionados.append(campos[indiceMaximo])
      rank.append(i+1)

      #Borrar los datos
      del campos[indiceMaximo]
      del scores[indiceMaximo]
      del indices[indiceMaximo]

    features=[nombreCamposSeleccionados,scoreSeleccionados,posicionCamposSeleccionados,rank]
    return features





################################################################################
################################################################################
#
#       FUNCIONES PARA TRANSFORMACION
#
################################################################################
################################################################################


  def estandarizar(self,nombresColumnas):
    
    return 0


  def convertirCategoricasToBinaria(self,nombresCategoricas):

    columnas=nombresCategoricas #lista de strings con nombres de las columnas
    columnasToDrop=self.getNombresNumericas()+self.nombreTarget

    df_NoNumericas=self.dataframe.drop(columns=columnasToDrop).copy()
    df_NoNumericas=pd.get_dummies(df_NoNumericas)

    self.quitarColumnas(nombresCategoricas)

    nombresBinarias=list(df_NoNumericas.columns)
    nombresBinarias.reverse()

    #Agregar las columnas creadas al inicio del dataframe original
    for columna in nombresBinarias:
      self.datagrame.insert(loc=0, column=columna, value=df_NoNumericas[columna])
    return None

  def convertirTargetToBinaria(self):
    targets=self.nombreTarget=[]
    self.nombreTarget=[]
    columnas=nombresCategoricas #lista de strings con nombres de las columnas
    return None





################################################################################
################################################################################
################################################################################
#       EXCLUSIVAS PARA EL PROBLEMA DE TERNIUM
#       EXCLUSIVAS PARA EL PROBLEMA DE TERNIUM
#       EXCLUSIVAS PARA EL PROBLEMA DE TERNIUM
################################################################################
################################################################################
################################################################################





################################################################################
################################################################################
#
#       FUNCIONES PARA GRAFICACION
#
################################################################################
################################################################################
################################################################################








#'''
# DESCRIPCION DE DATOS: Graficacion y resumen
# ##############################################################################
#'''

# Tratamiento de datos
################################################################################


# GRAFICACION DESCRIPTIVA - RESUMEN
# ==============================================================================

# GRAFICACION DESCRIPTIVA - RESUMEN
# ==============================================================================




In [ ]:
#   _____ _                   _____      _                  
#  / ____| |                 / ____|    | |                 
# | |    | | __ _ ___ ___   | (___   ___| |_ ___  ___  __ _ 
# | |    | |/ _` / __/ __|   \___ \ / _ \ __/ _ \/ __|/ _` |
# | |____| | (_| \__ \__ \   ____) |  __/ || (_) \__ \ (_| |
#  \_____|_|\__,_|___/___/  |_____/ \___|\__\___/|___/\__,_|
#                                                           
                                                                                                                                                                                             







'''
################################################################################
################################################################################
################################################################################
################################################################################


# ESTA CLASE ESTA PENSADA PARA HACER EL PROCESO DE ENTRENAMIENO Y EVALUACION DE
MODELOS DE MANERA MAS RAPIDA, PRACTICA Y ENTENDIBLE.


################################################################################
################################################################################
################################################################################
################################################################################
'''
#Clase para evaluar los modelos de entrenamiento
#Como parámetros se le deben de dar los features y el target, en forma de 
#dataframe o series.

class Setosa:

  #Estas de aqui son condiciones Booleanas, que por el momento no se han usado
  #pero que estan planeadas para mostrara avisos y mensajes de error.
  #Por ejenplo, mostrar un error si se quiere mostrar el reporte del arbol
  #pero aun no se ha entrenado el modelo
  arbolEntrenado=False
  regresionLogisticaEntrenado=False
  svmEntrenado=False

  #Los conjuntos de entrenamiento y de prueba son guardados como atributos
  #internos de la clase, de esta manera, cualquier modelo de clasificacion
  #dentro de la clase puede acceder a ellos sin tener que volver a crearlos
  x_train=[]
  x_test=[]
  y_train=[]
  y_test=[]

  #Las predicciones de cada modelo tambien se guardan como atributos de clase
  #con el objetivo de que al realizar los reportes de metricas no se tenga que
  #calcular nuevamente las predicciones ni que se agregen lineas de mas
  y_predicted_arbol=[]
  y_predicted_logreg=[]
  y_predicted_svm=[]


  #Asimismo, los modelos de clasificacion se guardan de forma interna como
  #atributos de la clase, de tal maneta que cualquier funcion miembro puede
  #puede acceder a ellos

  #PRE-CREACION DE MODELOS
  arbol= tree.DecisionTreeClassifier()
  logRec = LogisticRegression(solver='liblinear', random_state=0)

  #Para el svm
  pipeline= make_pipeline(StandardScaler(), SVC(gamma='auto'))
  svm=make_pipeline(StandardScaler(), SVC(gamma='auto'))


  #LA funcion que inicializa el objeto, se requieren como parametros, las
  #columnas de features y la columna del target, en forma de dataframe o serie
  #de Pandas
  def __init__(self,features,target):
    self.features=pd.DataFrame()
    self.target=pd.DataFrame()
    self.features=features.copy()
    self.target=target.copy()


  #Funcion que genera los conjuntos de entrenamiento y de prueba.
  #Como parametros se requieren el test size y el random state
  def generarSetsTrainTest(self,porcentajeTest,seed):
    x=self.features
    y=self.target
    self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(x, y, test_size = porcentajeTest, random_state = seed)
    return None


  #Con esta funcion se entrena el modelo de arbol de decisiones
  #como parametros se requiere el random state y el la profunidad maxima
  #Si el parametro de profundidad es menor o igual que cero, entonces se 
  #asume que no hay limite de profunidad
  def entrenarArbol(self,seed,profundidadMax):

    if (profundidadMax <= 0):
      self.arbol=tree.DecisionTreeClassifier(random_state=seed)
      self.arbol = self.arbol.fit(self.x_train, self.y_train)
      self.y_predicted_arbol=self.arbol.predict(self.x_test)
    else:
      self.arbol=tree.DecisionTreeClassifier(random_state=seed,max_depth=profundidadMax)
      self.arbol = self.arbol.fit(self.x_train, self.y_train)
      self.y_predicted_arbol=self.arbol.predict(self.x_test)

    return None


  #Como indica su nombre, este metodo entrena el modelo de regresion logistica
  #que hay internamente
  def entrenarRegresionLogistica(self):
    self.logRec.fit(self.x_train, self.y_train)
    self.y_predicted_logreg=self.logRec.predict(self.x_test)
    return None

  #Del mismo modo, esta funcion entrena el modelo SVM
  def entrenarSVM(self):
    self.svm=self.pipeline.fit(self.x_train, self.y_train)
    self.y_predicted_svm=self.svm.predict(self.x_test)
    return None



  #
  def evaluarModelo(self,nombreClasificador):

    if nombreClasificador=="arbol":
      clasificador=self.arbol
      y_predicted=self.y_predicted_arbol
    if nombreClasificador=="logreg":
      clasificador=self.logRec
      y_predicted=self.y_predicted_logreg
    if nombreClasificador=="svm":
      clasificador=self.svm
      y_predicted=self.y_predicted_svm

    #Matriz de confussion array
    matriz=confusion_matrix(self.y_test,y_predicted,labels=clasificador.classes_)
    
    #Accuracy
    accuracy=accuracy_score(self.y_test,y_predicted)

    #Precission
    precision=precision_score(self.y_test, y_predicted)

    #Recall (sensibilidad)
    recall=recall_score(self.y_test, y_predicted)

    #Specificity
    tn, fp, fn, tp = matriz.ravel()
    specificity = tn / (tn+fp)

    #F1-SCORE
    f1=f1_score(self.y_test, y_predicted)



    #AUC
    #NOTA: Este calculo no esta correcto, se debe corregir, la verdadera AUC se
    #muestra en la grafica generada de ROC
    #se calcula est metrica pero no se muestra en el reporte individual
    #se planea corregir luego
    if nombreClasificador=="svm":
      auc_score=roc_auc_score(self.y_train, clasificador.decision_function(self.x_train))
    else:
      auc_score=roc_auc_score(self.y_train, clasificador.predict_proba(self.x_train)[:, 1])


    #Todas las variables se guardan en una lista, y la funcion regresa dicha
    #lista con las metricas
    metricas=[accuracy,precision,recall,specificity,f1,auc_score]

    return metricas



  #Imprime un reporte con cada una de las metricas, en lugar de solo la lista
  #sin contexto con los valores.
  #Y tiene una mejor presentacion que el otro metodo que tiene el modulo de
  #sklearn
  def printReporteInd(self,nombreClasificador):
    metricas=[]
    nombre=""

    if nombreClasificador=="arbol":
      metricas=self.evaluarModelo("arbol")
      nombre="Arbol de decisiones"
    if nombreClasificador=="logreg":
      metricas=self.evaluarModelo("logreg")
      nombre="Regresion Logistica"
    if nombreClasificador=="svm":
      metricas=self.evaluarModelo("svm")
      nombre="Support Vector Machine"

    print("<<<    REPORTE INDIVIDUAL DEL MODELO: ",nombre," >>>> \n")
    print("Accuracy: ",metricas[0])
    print("Precision: ",metricas[1])
    print("Recall (sensibilidad): ",metricas[2])
    print("Especificidad: ",metricas[3])
    print("F1: ",metricas[4])
    #print("AUC: ",metricas[5])

    return None


  #Crea la matriz de confusion para cada uno de los modelos
  def mostrarMatrizConfusion(self,nombreClasificador):

    if nombreClasificador=="arbol":
      clasificador=self.arbol
      y_predicted=self.y_predicted_arbol
    if nombreClasificador=="logreg":
      clasificador=self.logRec
      y_predicted=self.y_predicted_logreg
    if nombreClasificador=="svm":
      clasificador=self.svm
      y_predicted=self.y_predicted_svm

    matriz=confusion_matrix(self.y_test,y_predicted,labels=clasificador.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=matriz,display_labels=clasificador.classes_)
    disp.plot()
    plt.show()
    plt.clf()

    return None


  def mostrarCurvaROC(self,nombreClasificador):

    if nombreClasificador=="arbol":
      clasificador=self.arbol
      y_predicted=self.y_predicted_arbol
      y_score = clasificador.fit(self.x_train, self.y_train).predict_proba(self.x_train)[:, 1]
    if nombreClasificador=="logreg":
      clasificador=self.logRec
      y_predicted=self.y_predicted_logreg
      y_score = clasificador.fit(self.x_train, self.y_train).predict_proba(self.x_train)[:, 1]
    if nombreClasificador=="svm":
      clasificador=self.svm
      y_predicted=self.y_predicted_svm
      y_score = clasificador.fit(self.x_train, self.y_train).decision_function(self.x_test)

    plot_roc_curve(clasificador, self.x_test, self.y_test)
    plt.show()
    plt.clf()

    return None





  #Funcion que aun no esta terminada, tiene el objetivo de crear un reporte
  #general de los 3 modelos, mostrando el mas eficiente y generando las
  #graficas de la matriz de confusion
  def reporteGeneral(self):
    return 0



  #Funcion para graficar la ROC, no se va a usar por ahora porque debe
  #ser corregida
  def mostrarCurvaROC_plus(self,nombreClasificador):

    if nombreClasificador=="arbol":
      clasificador=self.arbol
      y_predicted=self.y_predicted_arbol
      y_score = clasificador.fit(self.x_train, self.y_train).predict_proba(self.x_train)[:, 1]
    if nombreClasificador=="logreg":
      clasificador=self.logRec
      y_predicted=self.y_predicted_logreg
      y_score = clasificador.fit(self.x_train, self.y_train).predict_proba(self.x_train)[:, 1]
    if nombreClasificador=="svm":
      clasificador=self.svm
      y_predicted=self.y_predicted_svm
      y_score = clasificador.fit(self.x_train, self.y_train).decision_function(self.x_test)

    #CALCULAR LOS VALORES DE LA CURVA


    # Compute ROC curve and ROC area for each class
    #n_classes = self.y_train.shape[1]
    n_classes = 2

    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(self.y_test[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])    


    #CREAR LA FIGURA CON MATPLOTLIB
    
    plt.figure()
    lw = 2
    plt.plot(
        fpr[2],
        tpr[2],
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.2f)" % roc_auc[2],
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic example")
    plt.legend(loc="lower right")
    plt.show()
    plt.clf()
    return None


#FUNCION DESCARTADA PERO SE MANTIENE POR SI SE REQUIERE REGRESAR A UN INICIO
  def printReporteInd_Descartado(self,nombreClasificador):
    metricas=[]
    nombre=""

    if nombreClasificador=="arbol":
      metricas=self.evaluarArbol()
      nombre="Arbol de decisiones"
    if nombreClasificador=="logreg":
      metricas=self.evaluarRegresionLogistica()
      nombre="Regresion Logistica"
    if nombreClasificador=="svm":
      metricas=self.evaluarSVM()
      nombre="Support Vector Machine"

    print("<<<    REPORTE INDIVIDUAL DEL MODELO: ",nombre," >>>> \n")
    print("Accuracy: ",metricas[0])
    print("Precision: ",metricas[1])
    print("Recall (sensibilidad): ",metricas[2])
    print("Especificidad: ",metricas[3])
    print("F1: ",metricas[4])
    print("AUC: ",metricas[5])

    return None

In [ ]:

'''
################################################################################
################################################################################
################################################################################
################################################################################

En la siguiente seccion estan primero unas funciones adicionales, de ayuda que 
se usaran dentro de la clase Versicolor para limpieza, descripcion y ayuda jijiji

################################################################################
################################################################################
################################################################################
################################################################################
'''

def contarAtipicosColumna(columnaSeries,nombreColumna):
    columna=columnaSeries.copy()
    q1 = columna.quantile(0.25)
    q3 = columna.quantile(0.75)

    #Rango intercuartilico
    rango = q3-q1 

    limiteInferior  = q1-1.5*rango
    limiteSuperior = q3+1.5*rango

    #Se crea un dataframe/Serie
    data=pd.DataFrame(columna)
    df = data.loc[ (data[nombreColumna] < limiteInferior) & (data[nombreColumna] > limiteSuperior)  ]

    #Se regresa el numero de filas del dataframe
    return df.shape[0]

#FUNCION PARA ELIMINAR OUTLIERS DE UNA COLUMNA
def quitarAtipicosColumna(dataframe, columna):
    q1 = dataframe[columna].quantile(0.25)
    q3 = dataframe[columna].quantile(0.75)

    #Rango intercuartilico
    rango = q3-q1 

    limiteInferior  = q1-1.5*rango
    limiteSuperior = q3+1.5*rango

    #Se regresa un nuevo dataframe con sin los valores atipicos de cada columna
    #Se crea un dataframe en donde esas columnas estén dentro del limite inferior y mayor
    df = dataframe.loc[(dataframe[columna] > limiteInferior) & (dataframe[columna] < limiteSuperior)]

    #Se regresa el nuevo dataframe
    return df